## <font color=blue> Target (ground truth): NC county deaths

Need ...
To figure out what the "naive" answer is. It would be the county ratio multiplied by the state prediction. First, do this for 2021.




In [1]:
import pandas as pd

df = pd.read_csv('raw_data/nyt_us-counties-2021.csv')
# display(df.head(), len(df))

df.columns = map(lambda x: str(x).capitalize(), df.columns)

df = df[df.State=='North Carolina']
df = df.drop(['Cases','State'],axis=1)
df = df[df.County!="Unknown"]

## Re-format

df = df.rename({'Deaths':'True_county_deaths'},axis=1)

df['True_county_deaths'] = df['True_county_deaths'].astype('Int64')
df['Fips'] = df['Fips'].astype('Int64').astype('str')

df = df.reset_index(drop=True)

df_counties = df
df_counties

,Date,County,Fips,True_county_deaths
0,2021-01-01,Alamance,37001,117
1,2021-01-01,Alexander,37003,30
2,2021-01-01,Alleghany,37005,2
3,2021-01-01,Anson,37007,26
4,2021-01-01,Ashe,37009,27
...,...,...,...,...
36495,2021-12-31,Wayne,37191,332
36496,2021-12-31,Wilkes,37193,205
36497,2021-12-31,Wilson,37195,243
36498,2021-12-31,Yadkin,37197,90


<font color=blue> Get population of each county from covariates json. Just use whatever ones. 2019?

In [3]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)
print(len(d))



county_covs = []
for entry in d:
    if entry['state']=='NC':
        county_covs.append(entry)
print(len(county_covs))



df_pop = pd.DataFrame(columns=['Fips','Pop'])

for idx in range(len(county_covs)):
    pop = county_covs[idx]['population']['2019']
    fips = county_covs[idx]['fips']
    df_pop.loc[idx] = pd.Series({'Fips':str(fips), 'Pop':pop})
    
    
tot_pop = sum(df_pop.Pop)
tot_pop

df_pop['Pop_ratio'] = df_pop['Pop'].div(tot_pop)
df_pop

3142
100


,Fips,Pop,Pop_ratio
0,37037,74470,0.0071
1,37001,169509,0.016162
2,37057,167609,0.015981
3,37069,69685,0.006644
4,37155,130625,0.012455
...,...,...,...
95,37175,34385,0.003278
96,37171,71783,0.006844
97,37043,11231,0.001071
98,37055,37009,0.003529


In [4]:
dff = df_counties.merge(df_pop, on='Fips')
dff

,Date,County,Fips,True_county_deaths,Pop,Pop_ratio
0,2021-01-01,Alamance,37001,117,169509,0.016162
1,2021-01-02,Alamance,37001,117,169509,0.016162
2,2021-01-03,Alamance,37001,117,169509,0.016162
3,2021-01-04,Alamance,37001,117,169509,0.016162
4,2021-01-05,Alamance,37001,124,169509,0.016162
...,...,...,...,...,...,...
36495,2021-12-27,Yancey,37199,36,18069,0.001723
36496,2021-12-28,Yancey,37199,36,18069,0.001723
36497,2021-12-29,Yancey,37199,36,18069,0.001723
36498,2021-12-30,Yancey,37199,36,18069,0.001723


<font color=blue> Get state predictions for each week (?) or whatever.

In [10]:
df = pd.read_csv('processed_data/nc_state_projections.csv')

# Re-format dataframe ...
df.columns = map(lambda x: str(x).capitalize(), df.columns)
df = df[df.Type=='point']
df = df[df.Target=='1 wk ahead cum death']
df = df.drop(['Target','Forecast_date','Quantile','Type','Location'],axis=1)

df = df.rename({'Target_end_date':'Date','Value':'Pred_state_deaths'},axis=1)
df = df.reset_index(drop=True)
df_state_preds = df

# print(len(df_state_preds))
df_state_preds.head()

,Date,Pred_state_deaths
0,2021-02-06,9846
1,2021-02-13,10485
2,2021-02-20,11231
3,2021-02-27,11285
4,2021-03-06,11494


In [11]:
dfff = dff.merge(df_state_preds, on='Date')
dfff

,Date,County,Fips,True_county_deaths,Pop,Pop_ratio,Pred_state_deaths
0,2021-02-06,Alamance,37001,178,169509,0.016162,9846
1,2021-02-06,Alexander,37003,59,37497,0.003575,9846
2,2021-02-06,Alleghany,37005,4,11137,0.001062,9846
3,2021-02-06,Anson,37007,45,24446,0.002331,9846
4,2021-02-06,Ashe,37009,36,27203,0.002594,9846
...,...,...,...,...,...,...,...
3895,2021-12-25,Wayne,37191,332,123131,0.01174,19285
3896,2021-12-25,Wilkes,37193,198,68412,0.006523,19285
3897,2021-12-25,Wilson,37195,243,81801,0.007799,19285
3898,2021-12-25,Yadkin,37197,82,37667,0.003591,19285


In [6]:
dfff['Naive_county_deaths'] = dfff.apply(lambda x: x.Pop_ratio * x.Pred_state_deaths, axis=1)
dfff

,Date,County,Fips,True_county_deaths,Pop,Pop_ratio,Pred_state_deaths,Naive_county_deaths
0,2021-02-06,Alamance,37001,178,169509,0.016162,9846,159.131602
1,2021-02-06,Alexander,37003,59,37497,0.003575,9846,35.201421
2,2021-02-06,Alleghany,37005,4,11137,0.001062,9846,10.455189
3,2021-02-06,Anson,37007,45,24446,0.002331,9846,22.949408
4,2021-02-06,Ashe,37009,36,27203,0.002594,9846,25.537623
...,...,...,...,...,...,...,...,...
3895,2021-12-25,Wayne,37191,332,123131,0.01174,19285,226.407544
3896,2021-12-25,Wilkes,37193,198,68412,0.006523,19285,125.792797
3897,2021-12-25,Wilson,37195,243,81801,0.007799,19285,150.411866
3898,2021-12-25,Yadkin,37197,82,37667,0.003591,19285,69.260324


In [7]:
# dfff.to_csv('processed_data/naive_county_deaths.csv',index=False)

In [12]:
dfff[dfff.County=='Wake']

,Date,County,Fips,True_county_deaths,Pop,Pop_ratio,Pred_state_deaths
91,2021-02-06,Wake,37183,496,1111761,0.106002,9846
191,2021-02-13,Wake,37183,505,1111761,0.106002,10485
291,2021-02-20,Wake,37183,523,1111761,0.106002,11231
391,2021-02-27,Wake,37183,521,1111761,0.106002,11285
491,2021-03-06,Wake,37183,550,1111761,0.106002,11494
591,2021-05-08,Wake,37183,729,1111761,0.106002,12773
691,2021-05-15,Wake,37183,732,1111761,0.106002,12893
791,2021-05-22,Wake,37183,732,1111761,0.106002,12946
891,2021-05-29,Wake,37183,735,1111761,0.106002,13030
991,2021-06-05,Wake,37183,737,1111761,0.106002,13154
